In [1]:
import dask.dataframe as dd
import datetime
from pathlib import Path

seed=2020

In [2]:
# loading metadata file
filePath = '/'
df = dd.read_csv(filePath + '.csv').drop_duplicates()

# Pre-processing to filter out inconsistent data
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]

In [3]:
lista = []
# List all files in directory using pathlib
basepath = Path("")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)

In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file

In [5]:
# Create dataset with only Identifiers in 2019 
df_in_2019 = df[df.Identifier.isin(lista_FileName)]
# df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]

In [6]:
# Creation of datasets for 1 and 2 occupants
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 1) & (df_in_2019['Number of Remote Sensors'] >= 3)]

In [7]:
df2.compute().shape

(941, 22)

In [9]:
# Take all Identifiers of df2, to after open them
a=df2.loc[:,'Identifier'].compute().tolist()
lista=[]
for i in a:
    lista.append(mainPath+i+'.csv')

In [10]:
# Read all files in the df2['Identifier'] column
ff=dd.read_csv(lista, usecols = use_cols, parse_dates=['DateTime'], include_path_column=True)
ff['path']=ff['path'].str.replace('/', '')
ff['path']=ff['path'].str.replace('.csv', '')

In [11]:
# Manage of columns

ff['activation'] = ff.loc[:, ff.columns.str.contains('_Motion')].sum(axis=1) # sumo cantidad de rs en 1
ff['dayofweek']=ff['DateTime'].dt.day_name()
ff['Time'] = ff['DateTime'].dt.time
ff['Date'] = ff['DateTime'].dt.date

In [12]:
ff_columns = ['path', 'activation', 'dayofweek', 'Time','Date']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result['mean_activation']=result['activation']/(result['Number of Remote Sensors']+1) # +1 counting sensor of thermostat

In [13]:
# Select only the spring and summer rows
date_result =dd.to_datetime(result['Date'])
# Spring
start_date_sp = datetime.datetime.strptime('2019-03-21','%Y-%m-%d')
end_date_sp = datetime.datetime.strptime('2019-06-21','%Y-%m-%d')
result_sp = result[(date_result >= start_date_sp) & (date_result < end_date_sp)]
result_sp['season'] = 'spring'


In [14]:
# ----------------------Spring-------------------------
aa_spring=result_sp[['Identifier','dayofweek','Time','mean_activation']].groupby(['Identifier','dayofweek','Time']).mean()

In [15]:
pp_spring=dd.from_array(aa_spring.to_records()) 

In [16]:
pp_spring["TimeSTR"]=pp_spring["Time"].apply((lambda x: x.strftime('%H:%M')), meta='str')
aa2_spring = pp_spring.copy()
new_spring = aa2_spring["TimeSTR"]
# concatenating team with name column 
# overwriting name column 
aa2_spring["dayofweek"]= aa2_spring["dayofweek"].str.cat(new_spring, sep =", ") 

In [17]:
bb_spring = aa2_spring.categorize(columns='dayofweek')

In [18]:
table_spring = dd.pivot_table(bb_spring, values='mean_activation', index='Identifier', columns='dayofweek')

In [19]:
result_f_spring = dd.merge(df2,table_spring, on='Identifier').drop(['filename'],axis=1)
result_f_spring['season'] = 'spring'
#----------------------------------------------------------------------------------

In [20]:
(result_f_spring.compute()).to_csv('.csv')
